In [ ]:
all_text = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_EBA.txt', 'r').readlines()

In [ ]:
q_value_list = []
for line in all_text[1:]:
    q_value_list.append(float(line.split('\t')[-1].strip()))

In [ ]:
from pprint import pprint
print(len(all_text_b))
pprint([a.split('\t') for a in all_text_b])

In [ ]:
all_text_b = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_TDA_RandomPeaks.txt', 'r').readlines()


In [ ]:
q_value_list_b = []
for line in all_text_b[1:]:
    q_value_list_b.append(float(line.split('\t')[-1].strip()))

In [ ]:
import pylab as plt
plt.figure(figsize=(8,6))
plt.plot(q_value_list_b, q_value_list, label='random')
plt.plot([0,0.4], [0,0.4], 'k--')
plt.xlabel('True')
plt.ylabel('Estimated')
plt.legend()
plt.grid()

In [ ]:
#calculating the true q_values
con_peaks_all = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-GnpsDecoyConditionalPeaks.txt', 'r').readlines()
cond_peaks_scores = []
for line in con_peaks_all[1:]:
    cond_peaks_scores.append(float(line.split('\t')[-1].strip()))

In [ ]:
print(cond_peaks_scores)

In [ ]:
from pprint import pprint
pprint([a.split('\t') for a in con_peaks_all])

In [ ]:
calculate_q_value(cond_peaks_scores)

In [ ]:
"1S/C13H18ClNO/c1-9(15-13(2,3)4)12(16)10-6-5-7-11(14)8-10/h5-9,15H,1-4H3"=="1S/C13H18ClNO/c1-9(15-13(2,3)4)12(16)10-6-5-7-11(14)8-10/h5-9,15H,1-4H3"

In [ ]:
import sys
sys.path.append(r'C:\Users\Gosia\Desktop\FDR-Metabolomics\src\passatutto_parser.py')
import passatutto_parser as pp
pre_spectrums = pp.PassatuttoParser(r'C:\Users\Gosia\Downloads\MassbankOrbi').parse_folder()

In [ ]:
from matchms.importing.load_from_json import as_spectrum
spectrums = []
for i, s in enumerate( pre_spectrums ):
    spectrums.append(as_spectrum(s))
    if i and i % 10 == 0:
        print('processed %d'%i)

In [ ]:
from pprint import pprint
name_inchi = {}
for s in spectrums:
    name_inchi[s.get("compound_name")] = s.get("inchi")
    
pprint(name_inchi)

In [170]:
file = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps.txt', 'r').readlines()

def cut_inchi(inchi):
    return inchi.split("=")[1].split("-")[0]
    
from collections import namedtuple
Hit = namedtuple('Hit', ['query', 'score', 'false_hit', 'inchi'])

hits = []
for line in file[1:]:
    q,t,ti,s = line.split("\t")
    hit = cut_inchi(name_inchi[q]) != cut_inchi(ti)
    hits.append(Hit(q,s,hit,ti))
    



In [ ]:
print(calculate_q_value(hits))

In [177]:
random_q_vals = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_TDA_RandomPeaks.txt', 'r').readlines()
    
def cut_inchi(inchi):
    return inchi.rsplit('/',2)[1]
    
hits = []
name_q_val = {}
for line in random_q_vals[1:]:
    q,t,ti,s,q_val = line.split("\t")
    hit = cut_inchi(name_inchi[q]) != cut_inchi(ti)
    #hit = name_inchi[q] != ti
    name_q_val.setdefault(q, []).append( (q_val,ti) )
    hits.append(Hit(q,s,hit,ti))

In [178]:
from collections import Counter
c = Counter( h.query for h in hits )
pprint( c.most_common(3) )
print(len(hits))

[('Erythromycin', 2), ('Reserpine', 2), ('Adenosine', 2)]
163


In [179]:
import numpy as np
_,_,q_values = calculate_q_value(hits)
for qval in q_values:
    print(qval.qval, qval.false_hit)

0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.034482758620689655 True
0.034482758620689655 False
0.034482758620689655 False
0.034482758620689655 False
0.034482758620689655 False
0.034482758620689655 False
0.034482758620689655 False
0.034482758620689655 False
0.034482758620689655 False
0.034482758620689655 False
0.034482758620689655 False
0.06451612903225806 True
0.06451612903225806 False
0.09375 True
0.0975609756097561 True
0.0975609756097561 False
0.0975609756097561 False
0.0975609756097561 False
0.0975609756097561 False
0.0975609756097561 False
0.0975609756097561 False
0.0975609756097561 False
0.0975609756097561 False
0.11475409836065574 True
0.11475409836065574 True
0.11475409836065574 False
0.11475409836065574 False
0.11475409836065574 False
0.11475409836065574 False
0.11475409836065574 False
0.11475409836065574 True
0.11475409836065574 False
0.11475

In [180]:
QVal = namedtuple( 'QVal', [ 'query', 'qval', 'inchi', 'false_hit'])

def calculate_q_value(hits):
    hits.sort(key=lambda x: x.score, reverse=True)
    fdr_vals = []
    for i in range(len(hits)):
        #calculate the proportion of false hits so far
        fdr = sum([int(h.false_hit) for h in hits[:i+1]])/(i+1)
        fdr_vals.append(fdr)
    q_vals = [0 for f in fdr_vals]
    q_vals[-1] = fdr_vals[-1]
    # ensure non-decreasing q_value
    for i in range(len(q_vals)-2,0,-1):
        q_vals[i] = min(fdr_vals[i],q_vals[i+1])
    q_list = []
    for i,h in enumerate(hits):
        # Simon says: if not h.false_hit:
        q_list.append(QVal(h.query, q_vals[i], h.inchi, h.false_hit))
    return fdr_vals, q_vals, q_list

In [182]:
def take_smaller(x):
    if len(x) == 1:
        return float(x[0][0])
    smaller = 0
    for i, (n,_) in enumerate(x):
        if float(n) < float(x[smaller][0]):
            smaller = i
    return float(x.pop(smaller)[0])

count = 0
print('Passatutto,   Ours,      Are They the same?     Name')
for name,q_val,inchi,_ in q_values:
    print('%4f      %4f    %s            %s' %( take_smaller(name_q_val[name]), float(q_val), cut_inchi(inchi)==cut_inchi(name_inchi[name]),name))
    

Passatutto,   Ours,      Are They the same?     Name
0.000000      0.000000    True            Bupropion
0.000000      0.000000    True            Ketoprofen
0.000000      0.000000    True            Sotalol
0.000000      0.000000    True            Flufenamic acid
0.000000      0.000000    True            Moclobemide
0.000000      0.000000    True            Mefenamic acid
0.000000      0.000000    True            Gabapentin
0.000000      0.000000    True            Venlafaxine
0.000000      0.000000    True            Clarithromycin
0.000000      0.000000    True            Metoclopramide
0.000000      0.000000    True            Fluconazole
0.000000      0.000000    True            Terbutryn
0.000000      0.000000    True            Clozapine
0.000000      0.000000    True            1-(3-(Trifluoromethyl)phenyl)piperazine
0.000000      0.000000    True            Sulfathiazole
0.000000      0.000000    True            Amisulpride
0.000000      0.000000    True            Sulfapyrid

In [ ]:
# Aminopyrine example:
Query:  Aminopyrine InChI=1S/C13H17N3O/c1-10-12(14(2)3)13(17)16(15(10)4)11-8-6-5-7-9-11/h5-9H,1-4H3
Target: Amidopyrine InChI=1S/C13H17N3O/c1-10-12(14(2)3)13(17)16(15(10)4)11-8-6-5-7-9-11/h5-9H,1-4H3
    
# Sebuthylazine example
Query: Sebuthylazine 
Target: MLS001048955-01!2,2-dimethyl-4,10-dihydro-3H-pyrano[2,3-b]quinolin-5-one
InChI=1S/C9H16ClN5/c1-4-6(3)12-9-14-7(10)13-8(15-9)11-5-2/h6H,4-5H2,1-3H3,(H2,11,12,13,14,15)
InChI=1S/C14H15NO2/c1-14(2)8-7-10-12(16)9-5-3-4-6-11(9)15-13(10)17-14/h3-6H,7-8H2,1-2H3,(H,15,16)

In [ ]:
InChI=1S/C20H24O2/c1-12-10-14-15-4-5-18(22)20(15,3)9-7-16(14)19(2)8-6-13(21)11-17(12)19/h6,8,11,14-16H,1,4-5,7,9-10H2,2-3H3/t14-,15-,16-,19+,20-/m0/s1
InChI=1S/C20H24O2/c1-12-10-14-15-4-5-18(22)20(15,3)9-7-16(14)19(2)8-6-13(21)11-17(12)19/h6,8,11,14-16H,1,4-5,7,9-10H2,2-3H3